# Coursera data science capstone project

This notebook contains the capstone project for Cousera Data Science course

***
# BEGINNING OF PART 1

Retrieving postcodes, boroughs and neighbourhoods in Toronto and treating missing values and duplicated postcodes

In [182]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

## Reading the data with web scraping

We will get the postcode, borough and neighbourhood data of Toronto from Wikipedia

In [183]:
#making the request to the Wikipedia page that contains the data for Toronto postcodes, borough and neighbourhoods
url_toronto_postcodes = "http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page_request = requests.get(url_toronto_postcodes).text

#scraping the page to get the html table
page_html = BeautifulSoup(page_request,"lxml")
postcode_table = page_html.find("table", class_="wikitable sortable")

In [184]:
#reading the data from the html table
df_toronto_postcode = pd.read_html(str(postcode_table))[0]
df_toronto_postcode

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


## Preparing the data

### Removing rows with not assigned borough

We need to check the information that is present in the Borough column
This will be important to confirm that all rows with not assigned borough will be removed

In [185]:
#how many rows do we have with borough = Not assigned? 
df_toronto_postcode["Borough"].value_counts()

Not assigned        77
Etobicoke           45
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

In [186]:
#Removing rows with not assigned borough
df_toronto_postcode = df_toronto_postcode[df_toronto_postcode["Borough"] != "Not assigned"].copy()

#Confirming the remaining values for borough
df_toronto_postcode["Borough"].value_counts()

Etobicoke           45
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

In [187]:
#Reseting the index after droping rows
df_toronto_postcode.reset_index(drop=True, inplace=True)
df_toronto_postcode.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Replacing not assigned neighbourhood with the borough name

We first check which rows do not have an assigned neighbourhood

In [188]:
df_toronto_postcode[df_toronto_postcode["Neighbourhood"] == "Not assigned"]

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Not assigned


In [189]:
#For those rows we assign the borough name to the neighbourhood column
df_toronto_postcode.loc[df_toronto_postcode["Neighbourhood"] == "Not assigned", "Neighbourhood"] = df_toronto_postcode.loc[df_toronto_postcode["Neighbourhood"] == "Not assigned", "Borough"]

In [190]:
#Then we confirm that no neighbourhood remained with a not assigned value
df_toronto_postcode[df_toronto_postcode["Neighbourhood"] == "Not assigned"]

,Postcode,Borough,Neighbourhood


### Concatenating neighbourhoods from the same postcode

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, the M5A postcode is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma. The same is applied to all the postcodes.

In [191]:
#Let's define a function to retrieve all the categories present in the data from a given row's postcode
def get_neighbourhoods_for_postcode(row):
    neighbourhood_series = df_toronto_postcode.loc[df_toronto_postcode["Postcode"] == row["Postcode"], "Neighbourhood"]
    neighbourhood_list = neighbourhood_series.tolist()
    return ",".join(neighbourhood_list)

#We apply that function to all the rows in the data and place the results in a new column
df_toronto_postcode["Neighbourhood List"] = df_toronto_postcode.apply(get_neighbourhoods_for_postcode, axis=1)

In [192]:
#We then drop the old neighbourhood column
df_toronto_postcode.drop(["Neighbourhood"],axis=1,inplace=True)

In [193]:
#Then we remove the duplicate rows keeping only the first occurrence 
df_toronto_postcode.drop_duplicates(keep="first",inplace=True)

In [194]:
#Let's rename the columns and reset the index
df_toronto_postcode.columns = ["Postcode", "Borough", "Neighbourhood"]
df_toronto_postcode.reset_index(drop=True, inplace=True)
df_toronto_postcode.head(20)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [195]:
#Let's check the shape of the dataframe
df_toronto_postcode.shape

(103, 3)

# END OF PART 1
***
# BEGINNING OF PART 2

Getting the latitute and longitude of each post code

In [207]:
#Let's read the latitude and longitude of each postcode in Toronto from a csv file
df_coordinates = pd.read_csv("./Geospatial_Coordinates.csv")
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [211]:
#We have to put in our previous dataframe the latitude and longitude data
df_toronto_postcode["Latitude"] = df_coordinates["Latitude"].copy()
df_toronto_postcode["Longitude"] = df_coordinates["Longitude"].copy()
df_toronto_postcode.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.763573,-79.188711
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.770992,-79.216917
4,M7A,Queen's Park,Queen's Park,43.773136,-79.239476


# END OF PART 2
***